In [2]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np 
import argparse
from copy import deepcopy #Add Deepcopy for args
import visdom
vis = visdom.Visdom()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

Setting up a new session...


1.4.0
Populating the interactive namespace from numpy and matplotlib


/Users/jinsungpark/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['seed']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# 1. Data loading & Preprocessing

### 1.1 Data loading

In [3]:
cd /Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA

/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA


In [4]:
pwd #현재 경로 확인

'/Users/jinsungpark/Desktop/jupyter/Data_river/original02/HP_DA'

In [5]:
ls #현재경로에 있는 항목 확인

DA_Down_data.xlsx  DA_Down_log.xlsx   HP_Up_data.xlsx    HP_Up_log.xlsx


In [6]:
UpStream_data = pd.read_excel('HP_Up_log.xlsx')
DownStream_data = pd.read_excel('DA_Down_log.xlsx')

In [7]:
print(UpStream_data.columns)
# print(DownStream_data.columns)

Index(['Date', 'HP_DO', 'HP_BOD', 'HP_COD', 'HP_SS', 'HP_TN', 'HP_TP',
       'HP_Chl_a', 'HP_Flow', 'ChaCh_DO', 'ChaCh_BOD', 'ChaCh_COD', 'ChaCh_SS',
       'ChaCh_TN', 'ChaCh_TP', 'ChaCh_Chl_a', 'ChaCh_Flow', 'UiR_Rain',
       'UiR_Solar', 'HP_Temp', 'ChaCh_Temp'],
      dtype='object')


In [8]:
#날짜 인덱스화
UpData = UpStream_data.set_index('Date')
DownData = DownStream_data.set_index('Date')

In [9]:
UpData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HP_DO        265 non-null    float64
 1   HP_BOD       265 non-null    float64
 2   HP_COD       265 non-null    float64
 3   HP_SS        265 non-null    float64
 4   HP_TN        265 non-null    float64
 5   HP_TP        265 non-null    float64
 6   HP_Chl_a     265 non-null    float64
 7   HP_Flow      265 non-null    float64
 8   ChaCh_DO     265 non-null    float64
 9   ChaCh_BOD    265 non-null    float64
 10  ChaCh_COD    265 non-null    float64
 11  ChaCh_SS     265 non-null    float64
 12  ChaCh_TN     265 non-null    float64
 13  ChaCh_TP     265 non-null    float64
 14  ChaCh_Chl_a  265 non-null    float64
 15  ChaCh_Flow   265 non-null    float64
 16  UiR_Rain     265 non-null    float64
 17  UiR_Solar    265 non-null    float64
 18  HP_Temp      265 non-null    fl

In [10]:
#넣고싶은 상류 항목 컬럼 선택 - TP setting
UpData = UpData.iloc[:,[0,2,3,4,5,18,7,8,10,12,13,19,15,16]]
print(UpData.columns)

Index(['HP_DO', 'HP_COD', 'HP_SS', 'HP_TN', 'HP_TP', 'HP_Temp', 'HP_Flow',
       'ChaCh_DO', 'ChaCh_COD', 'ChaCh_TN', 'ChaCh_TP', 'ChaCh_Temp',
       'ChaCh_Flow', 'UiR_Rain'],
      dtype='object')


In [11]:
DownData.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 265 entries, 2013-01-08 to 2019-09-23
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DA_DO     265 non-null    float64
 1   DA_BOD    265 non-null    float64
 2   DA_COD    265 non-null    float64
 3   DA_SS     265 non-null    float64
 4   DA_TN     265 non-null    float64
 5   DA_TP     265 non-null    float64
 6   DA_Chl_a  265 non-null    float64
dtypes: float64(7)
memory usage: 16.6 KB


In [12]:
#알고싶은 하류 항목 컬럼 넘버 넣기('Date'항목이 인덱스화 돼서 컬럼 넘버가 -1씩 됨)
Colum = 5
print(DownData.columns[Colum])

DA_TP


### 1.2 Data Preprocessing(normalization)

In [13]:
from sklearn.preprocessing import MinMaxScaler

UpScaler = MinMaxScaler() #상류데이터용
DownScaler = MinMaxScaler() #하류데이터용

#나중에 결과를 DeNormalizing 하기 위해 나누어 사용 하였다.

def DeNormalize(Y, Data_name, column_num, Scaler_Type):
    
    data = Data_name
    Scaler = Scaler_Type
    
    _max = Scaler.data_max_[column_num] # 역정규화 하려는 데이터의 컬럼 번호
    _min = Scaler.data_min_[column_num] 
    
    X = Y*(_max-_min) + _min
    
    return X

In [14]:
#데이터 정규화
UpData = pd.DataFrame(UpScaler.fit_transform(UpData), columns=UpData.columns, index=UpData.index)
DownData = pd.DataFrame(DownScaler.fit_transform(DownData), columns=DownData.columns, index=DownData.index)

print(UpData.isna().sum())

HP_DO         0
HP_COD        0
HP_SS         0
HP_TN         0
HP_TP         0
HP_Temp       0
HP_Flow       0
ChaCh_DO      0
ChaCh_COD     0
ChaCh_TN      0
ChaCh_TP      0
ChaCh_Temp    0
ChaCh_Flow    0
UiR_Rain      0
dtype: int64


## 2.Data Preparation

In [15]:
class RiverDataset(Dataset):
    def __init__(self, UpData, DownData, x_frames, y_frames, start, end):
        
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = start
        self.end = end

        self.UpData = UpData[start:end]
        self.DownData = DownData[start:end]

    def __len__(self):
        return len(self.UpData) - (self.x_frames + self.y_frames) + 1
    #데이터를 전처리 할때 UpData와 DownData의 길이가 동일해짐(날짜를 동일한것만 추출해야 하므로), 따라서 전체길이는 둘중 하나를 사용

    def __getitem__(self, idx):
        idx += self.x_frames

        X = self.UpData.iloc[idx-self.x_frames:idx].values
        Y = self.DownData.iloc[idx:idx+self.y_frames].values
        
        return X, Y

# 3. Model Define

In [16]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers) #
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [17]:
# 정확도 : 예측확률을 100%로 봤을때 MAPE에 따른 오차비율을 빼줌 (100-MAPE) ##RMSE, MAPE 두개로 볼 수 있게
def MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def RMSE(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    return np.sqrt(mse)

In [19]:
def R2(y_true, y_pred):
    R2_score = r2_score(y_true, y_pred, multioutput='raw_values')
    return R2_score

# 4. Train, Validate, Test

In [20]:
def train(model, partition, optimizer, loss_fn, args):
    trainloader = DataLoader(partition['train'],
                             batch_size=args.batch_size,
                             shuffle=False, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    train_acc = 0.0
    train_loss = 0.0
    
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)#파이토치는 순서가 달라서 바꿔줌
        y_true = y[:, :, Colum].float().to(args.device)
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        pred.append(y_pred)
        true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(trainloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   

    train_loss = train_loss / len(trainloader)
    train_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    train_acc2 = R2(np.array(true_results), np.array(pred_results))
#     train_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    return model, train_loss, train_acc1[0], train_acc2[0]

In [21]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    val_acc = 0.0
    val_loss = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 (배치사이즈의 텐서들을 한개씩 추가해주기) ==== #
    for i in range(len(valloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================================================== #   
    
    val_loss = val_loss / len(valloader)
    val_acc1 = RMSE(np.array(true_results), np.array(pred_results))
    val_acc2 = R2(np.array(true_results), np.array(pred_results))
#     val_acc3 = (100 - MAPE(np.array(true_results), np.array(pred_results)))

    
    return val_loss, val_acc1[0], val_acc2[0]

In [22]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    model.eval()

    bat_siz = args.batch_size
    pred = []
    true = []
    pred_results = []
    true_results = []
    test_acc = 0.0
    
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, Colum].float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)

            pred.append(y_pred)
            true.append(y_true)

    # ==== test 데이터 시각화를 위해 x,y데이터 저장 ==== #
    for i in range(len(testloader)):
        tems1 = pred[i].view(bat_siz).cpu().detach().numpy()
        tems2 = true[i].view(bat_siz).cpu().detach().numpy()
        
        for j in range(bat_siz):
            value1 = np.exp(DeNormalize(tems1[j], DownData, Colum, DownScaler))
            value2 = np.exp(DeNormalize(tems2[j], DownData, Colum, DownScaler))
            
            pred_results.append(value1)
            true_results.append(value2)
    # ========================================== #   

    test_acc1 =  RMSE(np.array( true_results), np.array(pred_results))
    test_acc2 =  R2(np.array( true_results), np.array(pred_results))
#     test_acc3 =  (100 - MAPE(np.array( true_results), np.array(pred_results)))
    
    return test_acc1[0], test_acc2[0], pred_results, true_results

# 5. Experiment

In [23]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)
#     loss_fn = torch.nn.MSELoss() ##loss는 mse를 사용
#     loss_fn = nn.MSELoss()
    
    if args.loss == 'MSELoss':
        loss_fn = torch.nn.MSELoss()
        loss_fn = nn.MSELoss()
    elif args.loss == 'L1Loss':
        loss_fn = torch.nn.L1Loss()
        loss_fn = nn.L1Loss()
    elif args.loss == 'PoissonNLLLoss':
        loss_fn = torch.nn.PoissonNLLLoss()
        loss_fn = nn.PoissonNLLLoss()
    elif args.loss == 'KLDivLoss':
        loss_fn = torch.nn.KLDivLoss()
        loss_fn = nn.KLDivLoss()
    elif args.loss == 'BCELoss':
        loss_fn = torch.nn.BCELoss()
        loss_fn = nn.BCELoss()
    elif args.loss == 'BCEWithLogitsLoss':
        loss_fn = torch.nn.BCEWithLogitsLoss()
        loss_fn = nn.BCEWithLogitsLoss()
    else:
        raise ValueError('In-valid LossFuction choice')
    
    
    
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs_RMSE = []
    train_accs_R2 = []
    val_accs_RMSE = []
    val_accs_R2 = []
    axis = []
    # ===================================== #
    
    ## model starting point ##    
    ts = time.time()
    model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
    val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
    te = time.time()

    # ====== Add Epoch Data ====== #
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs_RMSE.append(train_acc_RMSE)
    val_accs_RMSE.append(val_acc_RMSE)
    train_accs_R2.append(train_acc_R2)
    val_accs_R2.append(val_acc_R2)
    # ============================ #

    # # ===== Visdom visualizing ================================================================================== #
    axis.append(0)
    
    plot1 = vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_loss'.format(num), legend=['train_loss','val_loss'], showlegend=True))
    
    plot2 = vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                     X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                     opts=dict(title='exp_{}_acc_RMSE'.format(num), legend=['train_acc','val_acc'], showlegend=True))
    # # =========================================================================================================== #
    
    print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
          .format(0, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
    
    for epoch in range(args.epoch-1):  # loop over the dataset multiple times
        
        ts = time.time()
        model, train_loss, train_acc_RMSE, train_acc_R2 = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc_RMSE, val_acc_R2 = validate(model, partition, loss_fn, args)
        te = time.time()

        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs_RMSE.append(train_acc_RMSE)
        val_accs_RMSE.append(val_acc_RMSE)
        train_accs_R2.append(train_acc_R2)
        val_accs_R2.append(val_acc_R2)
        # ============================ #

        # # ===== Visdom visualizing ============================================================================== #
        axis.append(epoch+1)
        
        vis.line(Y=torch.cat((torch.Tensor(train_losses).view(-1,1), torch.Tensor(val_losses).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot1, update='replace')
        
        vis.line(Y=torch.cat((torch.Tensor(train_accs_RMSE).view(-1,1), torch.Tensor(val_accs_RMSE).view(-1,1)), -1),
                 X=torch.cat((torch.Tensor(axis).view(-1,1), torch.Tensor(axis).view(-1,1)), -1),
                 win=plot2, update='replace')
        # # ====================================================================================================== #
        
        print('Epoch {}, Acc_RMSE(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch+1, train_acc_RMSE, val_acc_RMSE, train_loss, val_loss, te-ts))
        
    test_acc_RMSE, test_acc_R2, Pred_data, True_data = test(model, partition, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    
    result['train_accs_RMSE'] = train_accs_RMSE
    result['train_accs_R2'] = train_accs_R2
    result['val_accs_RMSE'] = val_accs_RMSE
    result['val_accs_R2'] = val_accs_R2
#     result['train_acc'] = train_acc
#     result['val_acc'] = val_acc
    result['test_RMSE'] = test_acc_RMSE
    result['test_R2'] = test_acc_R2
    
    result['test_pred'] = Pred_data
    result['test_true'] = True_data
    return result

# 6. LSTM Run

In [35]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
args.batch_size = 8 # 2~8
args.UpData = UpData
args.DownData = DownData
args.x_frames = 2
args.y_frames = 1

# ====== Model Capacity ===== #
args.input_dim = len(UpData.columns)
args.hid_dim = 32
args.n_layers = 4

# ====== Regularization ======= #
args.l2 = 0.0001
args.dropout = 0.1 
args.use_bn = False

# ====== Optimizer & Training ====== #
args.optim = 'Adam'  #SGD, RMSprop, Adam...
args.loss = 'MSELoss'#'MSELoss','L1Loss','PoissonNLLLoss','KLDivLoss','BCELoss','BCEWithLogitsLoss'
args.lr = 0.0001
args.epoch = 200


# ====== Experiment Variable ====== #
name_var1 = 'x_frames'
name_var2 = 'batch_size'
name_var3 = 'optim'
name_var4 = 'lr'
name_var5 = 'epoch'
list_var1 = [2]
list_var2 = [8]
list_var3 = ['Adam']
list_var4 = [0.0001]
list_var5 = [200,300,400]


num = 0 #초기화


trainset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2013-01-01', '2018-06-30')
valset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
testset = RiverDataset(args.UpData, args.DownData, args.x_frames, args.y_frames, '2018-07-01', '2019-12-31')
partition = {'train': trainset, 'val':valset, 'test':testset}


print('size of trainset :{}'.format(len(trainset)),
      'size of valset :{}'.format(len(valset)),
      'size of testset :{}'.format(len(testset)))

for var1 in list_var1:
    for var2 in list_var2:
        for var3 in list_var3:
            for var4 in list_var4:
                for var5 in list_var5:
                    num += 1
                    setattr(args, name_var1, var1)
                    setattr(args, name_var2, var2)
                    setattr(args, name_var3, var3)
                    setattr(args, name_var4, var4)
                    setattr(args, name_var5, var5)
                    print('\n exp_{}'.format(num))
                    print('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                          .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn))     

                    result = experiment(partition, deepcopy(args))

            #         print('train_acc_RMSE = {:2.2f}%, val_acc_RMSE = {:2.2f}%, test_RMSE = {:2.2f}%, test_R2 = {:2.2f}%'
            #               .format(result['train_acc_RMSE'],result['val_acc_RMSE'],result['test_RMSE'],result['test_R2']))

                    vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
                             .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
                             opts=dict(title='exp_{}_text'.format(num)))
                    # 만든 모델의 test 데이터 예측 시각화

                    predict = torch.Tensor(result['test_pred']).view(-1,1)
                    truth = torch.Tensor(result['test_true']).view(-1,1)
                    axis = torch.Tensor(range(len(result['test_pred']))).view(-1,1)

                    Y_axis = torch.cat((predict, truth), -1)
                    X_axis = torch.cat((axis, axis), -1)

                    vis.line(Y = Y_axis, X = X_axis, opts=dict(title='Result_exp_{}_RMSE[{:2.3f}]_R2[{:2.3f}]'.format(num,result['test_RMSE'],result['test_R2']),
                                                               legend=['predict','true'],
                                                               showlegend=True,
                                                               layoutopts = {'plotly': {'legend': {'x':0, 'y':0}}}))

print('All done')

size of trainset :214 size of valset :47 size of testset :47

 exp_1
loseFunc = MSELoss, optim=Adam, x_frames=2, n_layers=4, batch_size=8, hid_dim=32, epoch=200, lr=0.0001, l2=0.0001, dropout=0.1, use_bn=False
Epoch 0, Acc_RMSE(train/val): 0.05/0.05, Loss(train/val) 0.26715/0.23590. Took 0.27 sec
Epoch 1, Acc_RMSE(train/val): 0.05/0.05, Loss(train/val) 0.25462/0.22425. Took 0.27 sec
Epoch 2, Acc_RMSE(train/val): 0.05/0.05, Loss(train/val) 0.24032/0.20935. Took 0.27 sec
Epoch 3, Acc_RMSE(train/val): 0.05/0.05, Loss(train/val) 0.22451/0.19428. Took 0.28 sec
Epoch 4, Acc_RMSE(train/val): 0.05/0.04, Loss(train/val) 0.20758/0.17831. Took 0.28 sec
Epoch 5, Acc_RMSE(train/val): 0.05/0.04, Loss(train/val) 0.19057/0.16088. Took 0.27 sec
Epoch 6, Acc_RMSE(train/val): 0.05/0.04, Loss(train/val) 0.17146/0.14334. Took 0.28 sec
Epoch 7, Acc_RMSE(train/val): 0.04/0.04, Loss(train/val) 0.15104/0.12397. Took 0.28 sec
Epoch 8, Acc_RMSE(train/val): 0.04/0.04, Loss(train/val) 0.12850/0.10245. Took 0.29 se

Epoch 90, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03022/0.03430. Took 0.27 sec
Epoch 91, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03012/0.03419. Took 0.26 sec
Epoch 92, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03038/0.03395. Took 0.27 sec
Epoch 93, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.02884/0.03373. Took 0.27 sec
Epoch 94, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.02997/0.03353. Took 0.27 sec
Epoch 95, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03075/0.03319. Took 0.27 sec
Epoch 96, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03045/0.03289. Took 0.27 sec
Epoch 97, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.02917/0.03263. Took 0.27 sec
Epoch 98, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.02828/0.03237. Took 0.27 sec
Epoch 99, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.02875/0.03217. Took 0.27 sec
Epoch 100, Acc_RMSE(train/val): 0.03/0.02, Loss(train/val) 0.03055/0.03192. Took 0.27 sec
Epoch 101, Acc_RMSE(

Epoch 182, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02430/0.02400. Took 0.27 sec
Epoch 183, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02354/0.02396. Took 0.27 sec
Epoch 184, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02412/0.02389. Took 0.27 sec
Epoch 185, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02387/0.02384. Took 0.27 sec
Epoch 186, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02477/0.02392. Took 0.28 sec
Epoch 187, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02389/0.02387. Took 0.27 sec
Epoch 188, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02213/0.02392. Took 0.27 sec
Epoch 189, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02456/0.02409. Took 0.28 sec
Epoch 190, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02458/0.02391. Took 0.27 sec
Epoch 191, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02416/0.02377. Took 0.27 sec
Epoch 192, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02376/0.02381. Took 0.28 sec
Epoch 193,

Epoch 73, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03598/0.04418. Took 0.27 sec
Epoch 74, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03616/0.04407. Took 0.27 sec
Epoch 75, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03576/0.04397. Took 0.27 sec
Epoch 76, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03583/0.04385. Took 0.27 sec
Epoch 77, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03522/0.04378. Took 0.27 sec
Epoch 78, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03593/0.04361. Took 0.27 sec
Epoch 79, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03545/0.04348. Took 0.27 sec
Epoch 80, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03553/0.04333. Took 0.27 sec
Epoch 81, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03581/0.04317. Took 0.28 sec
Epoch 82, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03513/0.04301. Took 0.27 sec
Epoch 83, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03527/0.04285. Took 0.27 sec
Epoch 84, Acc_RMSE(tr

Epoch 165, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02319/0.02388. Took 0.28 sec
Epoch 166, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02305/0.02390. Took 0.27 sec
Epoch 167, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02303/0.02390. Took 0.27 sec
Epoch 168, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02383/0.02390. Took 0.27 sec
Epoch 169, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02327/0.02389. Took 0.27 sec
Epoch 170, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02344/0.02390. Took 0.27 sec
Epoch 171, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02344/0.02390. Took 0.27 sec
Epoch 172, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02291/0.02393. Took 0.27 sec
Epoch 173, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02260/0.02392. Took 0.27 sec
Epoch 174, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02286/0.02392. Took 0.28 sec
Epoch 175, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02295/0.02391. Took 0.27 sec
Epoch 176,

Epoch 257, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02154/0.02408. Took 0.27 sec
Epoch 258, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02237/0.02407. Took 0.27 sec
Epoch 259, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02215/0.02407. Took 0.27 sec
Epoch 260, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02210/0.02406. Took 0.28 sec
Epoch 261, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02168/0.02406. Took 0.27 sec
Epoch 262, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02205/0.02408. Took 0.27 sec
Epoch 263, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02169/0.02412. Took 0.27 sec
Epoch 264, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02214/0.02411. Took 0.27 sec
Epoch 265, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02111/0.02411. Took 0.27 sec
Epoch 266, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02214/0.02411. Took 0.27 sec
Epoch 267, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02187/0.02412. Took 0.27 sec
Epoch 268,

Epoch 48, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03813/0.04643. Took 0.27 sec
Epoch 49, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03827/0.04642. Took 0.27 sec
Epoch 50, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03813/0.04640. Took 0.27 sec
Epoch 51, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03824/0.04641. Took 0.27 sec
Epoch 52, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03804/0.04641. Took 0.28 sec
Epoch 53, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03789/0.04640. Took 0.28 sec
Epoch 54, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03822/0.04640. Took 0.27 sec
Epoch 55, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03828/0.04639. Took 0.27 sec
Epoch 56, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03775/0.04638. Took 0.27 sec
Epoch 57, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03837/0.04638. Took 0.27 sec
Epoch 58, Acc_RMSE(train/val): 0.03/0.03, Loss(train/val) 0.03761/0.04639. Took 0.27 sec
Epoch 59, Acc_RMSE(tr

Epoch 140, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02437/0.02353. Took 0.27 sec
Epoch 141, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02385/0.02352. Took 0.27 sec
Epoch 142, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02413/0.02353. Took 0.27 sec
Epoch 143, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02351/0.02351. Took 0.27 sec
Epoch 144, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02370/0.02353. Took 0.27 sec
Epoch 145, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02384/0.02351. Took 0.27 sec
Epoch 146, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02336/0.02351. Took 0.27 sec
Epoch 147, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02400/0.02354. Took 0.27 sec
Epoch 148, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02383/0.02355. Took 0.27 sec
Epoch 149, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02314/0.02355. Took 0.27 sec
Epoch 150, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02443/0.02357. Took 0.27 sec
Epoch 151,

Epoch 232, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02275/0.02376. Took 0.27 sec
Epoch 233, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02274/0.02378. Took 0.27 sec
Epoch 234, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02293/0.02380. Took 0.27 sec
Epoch 235, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02279/0.02380. Took 0.27 sec
Epoch 236, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02260/0.02381. Took 0.27 sec
Epoch 237, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02369/0.02381. Took 0.27 sec
Epoch 238, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02273/0.02383. Took 0.27 sec
Epoch 239, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02313/0.02383. Took 0.27 sec
Epoch 240, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02314/0.02384. Took 0.28 sec
Epoch 241, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02273/0.02383. Took 0.28 sec
Epoch 242, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02266/0.02384. Took 0.27 sec
Epoch 243,

Epoch 324, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02161/0.02385. Took 0.28 sec
Epoch 325, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02228/0.02386. Took 0.27 sec
Epoch 326, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02170/0.02384. Took 0.27 sec
Epoch 327, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02178/0.02384. Took 0.27 sec
Epoch 328, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02211/0.02383. Took 0.27 sec
Epoch 329, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02169/0.02383. Took 0.27 sec
Epoch 330, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02174/0.02382. Took 0.27 sec
Epoch 331, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02156/0.02381. Took 0.27 sec
Epoch 332, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02164/0.02382. Took 0.27 sec
Epoch 333, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02182/0.02383. Took 0.28 sec
Epoch 334, Acc_RMSE(train/val): 0.02/0.02, Loss(train/val) 0.02241/0.02382. Took 0.28 sec
Epoch 335,

In [25]:
print('Current CPU random seed:',torch.initial_seed())
# print('Current CUDA random seed:',torch.cuda.initial_seed())

Current CPU random seed: 666


In [ ]:
# vis.text('loseFunc = {}, optim={}, x_frames={}, n_layers={}, batch_size={}, hid_dim={}, epoch={}, lr={}, l2={}, dropout={}, use_bn={}'
#                              .format(args.loss,args.optim,args.x_frames,args.n_layers,args.batch_size,args.hid_dim,args.epoch,args.lr,args.l2,args.dropout,args.use_bn),
#                              opts=dict(title='exp_{}_text'.format(num)))